In [1]:
class axis:
    def __init__(s,a,b):
        s.a = min(a,b)
        s.b = max(a,b)
    def print(s, c):
        l = ""
        l += c
        l += " "
        for i in range(s.b+1):
            if i >= s.a and i <= s.b: 
                l += "#"
            else:
                l += " "
        print(l)
    def str(s):
        return str(s.a)+"-"+str(s.b)
    def identical(s,o):
        return s.a == o.a and s.b == o.b
    def seperate(s, o):
        return s.b < o.a or o.b < s.a 
    def touch(s, o):
        return not s.seperate(o)
    def length(s):
        return s.b-s.a+1
    def shatter(s,o):
        ro = []
        ri = []
        #r.append(axis(s.a,s.b))
        if s.seperate(o):
            ro.append(axis(o.a,o.b))
        else:
            if o.a < s.a:
                ro.append(axis(o.a,s.a-1))
            ri.append(axis(max(s.a,o.a), min(s.b,o.b)))
            if o.b > s.b:
                ro.append(axis(s.b+1,o.b))
        return ri,ro
            
#test
for j in range(3):
    a = axis(2,2+2-j)
    for i in range(8):        
        b = axis(i,i+j)
        print()
        a.print("A")
        b.print("B")

        ri, ro = a.shatter(b)
        for x in ri:
            x.print("I")
        for x in ro:
            x.print("O")
    



A   ###
B #
O #

A   ###
B  #
O  #

A   ###
B   #
I   #

A   ###
B    #
I    #

A   ###
B     #
I     #

A   ###
B      #
O      #

A   ###
B       #
O       #

A   ###
B        #
O        #

A   ##
B ##
O ##

A   ##
B  ##
I   #
O  #

A   ##
B   ##
I   ##

A   ##
B    ##
I    #
O     #

A   ##
B     ##
O     ##

A   ##
B      ##
O      ##

A   ##
B       ##
O       ##

A   ##
B        ##
O        ##

A   #
B ###
I   #
O ##

A   #
B  ###
I   #
O  #
O    #

A   #
B   ###
I   #
O    ##

A   #
B    ###
O    ###

A   #
B     ###
O     ###

A   #
B      ###
O      ###

A   #
B       ###
O       ###

A   #
B        ###
O        ###


In [2]:
class rect:
    def __init__(s, x0, y0, x1, y1):
        s.x = axis(x0,x1)
        s.y = axis(y0,y1)
    
    def touch(s, o):
        return s.x.touch(o.x) and s.y.touch(o.y) 
    
    def area(s):
        return s.x.length() * s.y.length()
    def str(s):
        return "x"+s.x.str()+" y"+s.y.str()
    def merge(s, o):
        mergable = False
        if s.x.identical(o.x):
            if s.y.b + 1 == o.y.a or s.y.a - 1 == o.y.b:
                mergable = True
        elif s.y.identical(o.y):
            if s.x.b + 1 == o.x.a or s.x.a - 1 == o.x.b:
                mergable = True
        if mergable:
            r = rect(min(s.x.a, o.x.a), min(s.y.a, o.y.a), max(s.x.b, o.x.b), max(s.y.b, o.y.b))
            #print(s.str(), "+", o.str(), "=", r.str())
            return r
        #print(s.str(), " fail ", o.str())
        
        return None
    
    def shatter(s, o):
        r_in = []
        r_out = []
        
        x_in, x_out = s.x.shatter(o.x)
        y_in, y_out = s.y.shatter(o.y)
        
        for rx in x_in:
            for ry in y_in:
                r_in.append(rect(rx.a, ry.a, rx.b, ry.b))
        
        for rx in x_in:
            for ry in y_out:
                r_out.append(rect(rx.a, ry.a, rx.b, ry.b))
        for rx in x_out:
            for ry in y_in:
                r_out.append(rect(rx.a, ry.a, rx.b, ry.b))
        for rx in x_out:
            for ry in y_out:
                r_out.append(rect(rx.a, ry.a, rx.b, ry.b))
        
        
        
        
        return r_in, r_out    


In [ ]:
cmds = []
for l in open("06.txt").readlines():
    l=l.strip().split(" ")
    op=l[-4][:2]
    x0,y0=[int(x) for x in l[-3].split(",")]
    x1,y1=[int(x) for x in l[-1].split(",")]
    cmds.append((op, x0, y0, x1, y1))


def simplify(l):
    while True:
        #print("simplifying ",len(l))
        new_l = []
        ignore = []
        for i in range(len(l)):
            for j in range(len(l)):
                if i < j and j not in ignore and i not in ignore:
                    c = l[i].merge(l[j])
                    if c != None:
                        new_l.append(c)
                        ignore.append(i)
                        ignore.append(j)
            if i not in ignore:
                new_l.append(l[i])
        l = new_l
        if len(ignore) == 0:
            return l
        #print(len(l), ignore)
    

on = []
off = []
off.append(rect(0,0,999,999))
i = 0
for op, x0, y0, x1, y1 in cmds:
    #print(op, x0, y0, x1, y1)
    n = rect(x0, y0, x1, y1)
    
    new_on = []
    new_off = []
    
    for r in on:
        rin, rout = n.shatter(r)
        for r in rout:
            new_on.append(r)
        for r in rin:
            if op == "on":
                new_on.append(r)
            elif op == "of":
                new_off.append(r)
            else:
                new_off.append(r)
    
    for r in off:
        rin, rout = n.shatter(r)
        for r in rout:
            new_off.append(r)
        for r in rin:
            if op == "on":
                new_on.append(r)
            elif op == "of":
                new_off.append(r)
            else:
                new_on.append(r)
    
    new_on = simplify(new_on)
    new_off = simplify(new_off)
    
    on = new_on
    off = new_off
    
    i+=1
    
    if i % 100 == 0:
        print(i, op, len(on), len(off))
    
total_on = 0
for r in on:
    total_on += r.area()
total_off = 0
for r in off:
    total_off += r.area()
assert(total_on+total_off==1000000)




    

10 on 12 14
20 of 40 53
30 of 80 71
40 to 111 105
50 of 92 154
60 of 126 182
70 to 162 155
80 to 135 204
90 to 199 197
100 of 223 224
110 on 289 120
120 on 182 224
130 to 234 225
140 on 271 233
150 of 254 294
160 on 306 291


In [ ]:
grid = []
N=1000
for y in range(N):
    l = []
    for x in range(N):
        l.append(0)
    grid.append(l)
    
for op, x0, y0, x1, y1 in cmds:
    for y in range(y0,y1+1,1):
        for x in range(x0,x1+1,1):
            if op == "on":
                grid[y][x]+=1
            if op == "to":
                grid[y][x]+=2
            if op == "of":
                grid[y][x]=max(0,grid[y][x]-1)
                
print(sum([sum(x) for x in grid]))
                

In [ ]:
grid = []
N=1000
for y in range(N):
    l = []
    for x in range(N):
        l.append(0)
    grid.append(l)
    
for op, x0, y0, x1, y1 in cmds:
    for y in range(y0,y1+1,1):
        for x in range(x0,x1+1,1):
            if op == "on":
                grid[y][x]=1
            if op == "of":
                grid[y][x]=0
            if op == "to":
                grid[y][x]=1-grid[y][x]
                
print(sum([sum(x) for x in grid]))
                